## Example: Reading from a SQLite database

Lets begin by creating a small SQLite local database file.

Run the _"Creating the SQLite database"_ cell to create a new SQLite3 database with the filename you choose. You can double-click it to see the source code if you want.

In [ ]:
#@title Creating the SQLite database
import sqlite3

databse_file = "db/moon-phases.db" #@param {type:"string"}

with sqlite3.connect(databse_file) as db:
  cursor = db.cursor()

  # Create the moon_phases table.
  cursor.execute('''
    CREATE TABLE IF NOT EXISTS moon_phases (
      id INTEGER PRIMARY KEY,
      phase_emoji TEXT NOT NULL,
      peak_datetime DATETIME NOT NULL,
      phase TEXT NOT NULL)''')

  # Truncate the table if it's already populated.
  cursor.execute('DELETE FROM moon_phases')

  # Insert some sample data.
  insert_moon_phase = 'INSERT INTO moon_phases(phase_emoji, peak_datetime, phase) VALUES(?, ?, ?)'
  cursor.execute(insert_moon_phase, ('🌕', '2017-12-03 15:47:00', 'Full Moon'))
  cursor.execute(insert_moon_phase, ('🌗', '2017-12-10 07:51:00', 'Last Quarter'))
  cursor.execute(insert_moon_phase, ('🌑', '2017-12-18 06:30:00', 'New Moon'))
  cursor.execute(insert_moon_phase, ('🌓', '2017-12-26 09:20:00', 'First Quarter'))
  cursor.execute(insert_moon_phase, ('🌕', '2018-01-02 02:24:00', 'Full Moon'))
  cursor.execute(insert_moon_phase, ('🌗', '2018-01-08 22:25:00', 'Last Quarter'))
  cursor.execute(insert_moon_phase, ('🌑', '2018-01-17 02:17:00', 'New Moon'))
  cursor.execute(insert_moon_phase, ('🌓', '2018-01-24 22:20:00', 'First Quarter'))
  cursor.execute(insert_moon_phase, ('🌕', '2018-01-31 13:27:00', 'Full Moon'))

  # Query for the data in the table to make sure it's populated.
  cursor.execute('SELECT * FROM moon_phases')
  for row in cursor.fetchall():
    print(row)

We could use a `FlatMap` transform to receive a SQL query and `yield` each result row, but that would mean creating a new database connection for each query. If we generated a large number of queries, creating that many connections could be a bottleneck.

It would be nice to create the database connection only once for each worker, and every query could use the same connection if needed.

We can use a
[custom `DoFn` transform](https://beam.apache.org/documentation/transforms/python/elementwise/pardo/#example-3-pardo-with-dofn-methods)
for this. It allows us to open and close resources, like the database connection, only _once_ per `DoFn` _instance_ by using the `setup` and `teardown` methods.

> ℹ️ It should be safe to _read_ from a database with multiple concurrent processes using the same connection, but only one process should be _writing_ at once.

In [26]:
import apache_beam as beam
import sqlite3

class SQLiteSelect(beam.DoFn):
  def __init__(self, database_file):
    self.database_file = database_file
    self.connection = None

  def setup(self):
    self.connection = sqlite3.connect(self.database_file)

  def process(self, query):
    table, columns = query
    cursor = self.connection.cursor()
    cursor.execute(f"SELECT {','.join(columns)} FROM {table}")
    for row in cursor.fetchall():
      yield dict(zip(columns, row))

  def teardown(self):
    self.connection.close()

class SelectFromSQLite(beam.PTransform):
  def __init__(self, database_file, queries):
    self.database_file = database_file
    self.queries = queries

  def expand(self, pcollection):
    return (
        pcollection
        | 'Create None' >> beam.Create(queries)
        | 'SQLite SELECT' >> beam.ParDo(SQLiteSelect(self.database_file))
    )

database_file = 'db/moon-phases.db'
queries = [
    # (table_name, [column1, column2, ...])
    ('moon_phases', ['phase_emoji', 'peak_datetime', 'phase']),
    ('moon_phases', ['phase_emoji', 'phase']),
]
with beam.Pipeline() as pipeline:
  (
      pipeline
      | 'Read from SQLite' >> SelectFromSQLite(database_file, queries)
      | 'Keep only full moon' >> beam.Filter(lambda row: 'Full' in row.get('phase'))
      | 'Keep only 2018 or nulls' >> beam.Filter(lambda row: '2018' in row.get('peak_datetime', '2018'))
      | 'Print rows' >> beam.Map(print)
  )

{'phase_emoji': '🌕', 'peak_datetime': '2018-01-02 02:24:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'peak_datetime': '2018-01-31 13:27:00', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
{'phase_emoji': '🌕', 'phase': 'Full Moon'}
